#  Setup: Display options

In [1]:

#  Setting display options 

import pandas as pd
   #
pd.set_option("display.width", 480)

#  Sets horizontal scroll for wide outputs
#
from IPython.display import display, HTML
display(HTML(""))

from tabulate import tabulate

print("--")


--


# Setup: Connectivity

In [2]:

from katana import remote

my_client = remote.Client()

print(my_client)


In [3]:

NUM_PARTITIONS  = 3
   #
DB_NAME         = "my_db"
GRAPH_NAME      = "my_graph"

print("--")


--


In [4]:

#  DELETE ALL GRAPHS

for l_database in my_client.databases():
   for l_graph in my_client.get_database(name=l_database.name).graphs_in_database():
      l_handle=my_client.get_database(name=l_database.name).get_graph_by_id(id=l_graph.graph_id)
      l_handle.delete()

for l_graph in my_client.graphs():
   print("GRAPH ID: ", l_graph.graph_id, "      GRAPH Version: ", l_graph.version)

print("--")


--


In [ ]:

#  DELETE ALL DATABASES

for l_database in my_client.databases():
   if (l_database.name != "default"):
      my_client.get_database(name=l_database.name).delete_database()
      print("--")

for l_database in my_client.databases():
   print("DB ID: ", l_database.database_id, "     DB Name: ", l_database.name)


In [ ]:

#  CREATE DATABASE

my_database = my_client.create_database(name=DB_NAME)

print(my_database.database_id)


In [5]:

#  CREATE GRAPH
#
my_graph = my_client.get_database(name=DB_NAME).create_graph(name=GRAPH_NAME, num_partitions=NUM_PARTITIONS)

print(my_graph)


<_Graph my_graph, 2DhpYfrisxBadjxZgmaMmskD7gXWty3uyeKDbGuKFAaS, 0>


# Step 01:  Process CSV files ..

In [15]:

#  df_data      = pd.read_csv("gs://farrell-bucket/mongodb/21_users.txt")

df_data      = pd.read_csv("/home/jovyan/work/My_KG_NoteBooks/P1_Prospects/10_DataBricks_Google/10_Data/22_OncologyCallNotes.txt", header = "infer", delimiter = "|")

print(len(df_data))

l_cntr = 0
   #
for l_each in df_data.itertuples():
   
   print(l_each)





152
Pandas(Index=0, id='4001', medical_specialty='Hematology - Oncology', sample_name='3-Dimensional Simulation', description='3-Dimensional Simulation. This patient is undergoing 3-dimensionally planned radiation therapy in order to adequately target structures at risk while diminishing the degree of exposure to uninvolved adjacent normal structures.', body="This patient is undergoing 3-dimensionally planned radiation therapy in order to adequately target structures at risk while diminishing the degree of exposure to uninvolved adjacent normal structures. This optimizes the chance of controlling tumor while diminishing the acute and long-term side effects. With conformal 3-dimensional simulation, there is extended physician, therapist, and dosimetrist effort and time expended. The patient is initially taken into a conventional simulator room where appropriate markers are placed and the patient is positioned and immobilized. Preliminary filed sizes and arrangements, including gantry an

In [ ]:

#  In a Web browser
#     
#     https://console.cloud.google.com/storage/browser/a9187b90-cluster/kind-mullet/Paypal_20M_1B/
#

from google.cloud import storage


l_client      = storage.Client()

l_bucket_str  = "a9187b90-cluster"
   #
l_bucket      = l_client.bucket(l_bucket_str)


#  Just nodes
#
l_nodes = []
   #
for l_each in l_bucket.list_blobs(prefix = "kind-mullet/Paypal_20M_1B/N"):
   print(l_each.name)
      #
   l_nodes.append("gs://" + l_bucket_str + "/" + l_each.name)

print(l_nodes)


print("--")


#  Just edges
#
l_edges       = []
   #
l_cntr        = 0                                   #  We'll use these next two lines for testing
l_edges_small = []
   #
for l_each in l_bucket.list_blobs(prefix = "kind-mullet/Paypal_20M_1B/f"):
   print(l_each.name)
      #
   l_edges.append("gs://" + l_bucket_str + "/" + l_each.name)
      #
   l_cntr += 1
   if (l_cntr < 3):
      l_edges_small.append("gs://" + l_bucket_str + "/" + l_each.name)

print(l_edges)
print(l_edges_small)


print("--")

#  Sample output,
#
#     kind-mullet/Paypal_20M_1B/NodeFile_0
#     ['gs://a9187b90-cluster/kind-mullet/Paypal_20M_1B/NodeFile_0']
#     --
#     kind-mullet/Paypal_20M_1B/fileAssetA_0
#     kind-mullet/Paypal_20M_1B/fileAssetA_1
#     kind-mullet/Paypal_20M_1B/fileAssetA_2
#     kind-mullet/Paypal_20M_1B/fileAssetA_3
#     kind-mullet/Paypal_20M_1B/fileAssetA_4
#     kind-mullet/Paypal_20M_1B/fileAssetA_5
#     kind-mullet/Paypal_20M_1B/fileAssetB_0
#     kind-mullet/Paypal_20M_1B/fileAssetB_1
#     kind-mullet/Paypal_20M_1B/fileAssetB_2
#     kind-mullet/Paypal_20M_1B/fileAssetB_3
#     kind-mullet/Paypal_20M_1B/fileFriendA_0
#     kind-mullet/Paypal_20M_1B/fileFriendB_0
#     kind-mullet/Paypal_20M_1B/fileFriendB_1
#     ['gs://a9187b90-cluster/kind-mullet/Paypal_20M_1B/fileAssetA_0', 'gs://a9187b90-cluster/kind-mullet/Paypal_20M_1B/fileAssetA_1', 'gs://a9187b90-cluster/kind-mullet/P  ...
#     ['gs://a9187b90-cluster/kind-mullet/Paypal_20M_1B/fileAssetA_0', 'gs://a9187b90-cluster/kind-mullet/Paypal_20M_1B/fileAssetA_1']
#     --


#  Step 02: Read CSVs into Dask DataFrame

In [5]:

import dask.dataframe as dd
import numpy as np

print("--")


--


In [ ]:
%%time

#  Just nodes

#  Sample data,
#
#     Sent|counterparty|cntAppr|cntDecl|cntRiskDecl1|amtRiskDecl|cntRiskDecl2|sumRcvd|cntRcvd|cntDist|cntDecl2
#     0|N|N|N|N|XbRqfTcGJL9fxmNp4simTykwKr|qhBymw13PneK5hpntNVwjNwaYklX8P379JZo|Y|416|0|inactive|1941730454454|0|2018-01-24T01:09:18|9.07632e+09|1419493889|1894897856|2676929303.1430|6614868331.8687|8283173087.0037|6020979256.1980|1984359695|5077917542.6596|1923699935|1948138778|4876422147.1566|

df_nodes = dd.read_csv(
   l_nodes,
   delimiter = "|",
   dtype = {
      "Sent"          : np.dtype("O"),                         #  Change these if you wish,  Numpy datatypes
      "counterparty"  : np.dtype("O"),
      "cntAppr"       : np.dtype("O"),
      "cntDecl"       : np.dtype("O"),
      "cntRiskDecl1"  : np.dtype("O"),
      "amtRiskDecl"   : np.dtype("O"),
      "cntRiskDecl2"  : np.dtype("O"),
      "sumRcvd"       : np.dtype("O"),
      "cntRcvd"       : np.dtype("O"),
      "cntDist"       : np.dtype("O"),
      "cntDecl2"      : np.dtype("O"),
      })

print(len(df_nodes))

print("--")

#  Sample output,
#
#     20000000
#     --
#     CPU times: user 3min, sys: 18 s, total: 3min 18s
#     Wall time: 2min 48s


In [6]:
%%time

#  Just edges

#  Sample data,
#
#     Account1.id|Account2.id|type|date
#     16089173|16089175|FriendConB|2018-10-10T11:20:05

df_edges = dd.read_csv(
    
   #  l_edges,                                                   #  This would be all files, at once, in memory
   l_edges_small,
    
   delimiter = "|",
   dtype = {
      "Account1.id"   : np.dtype("O"),
      "Account2.id"   : np.dtype("O"),
      "type"          : np.dtype("O"),                           #  These next two column names are keywords in most worlds -- bad idea
      "date"          : np.dtype("O"),
      })

print(len(df_edges))

print("--")

#  Sample output,
#
#     200000000
#     --
#     CPU times: user 4min 25s, sys: 31.2 s, total: 4min 56s
#     Wall time: 4min 23s


200000000
--
CPU times: user 4min 25s, sys: 31.2 s, total: 4min 56s
Wall time: 4min 23s
